In [1]:
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [2]:
url = 'https://www.jobkorea.co.kr/'
keyword = '딥러닝'

browser = webdriver.Chrome()
browser.get(url)

In [3]:
search = browser.find_element(By.CLASS_NAME, 'smKey')
search.find_element(By.TAG_NAME, 'input').send_keys('딥러닝')
search_btn = browser.find_element(By.CLASS_NAME, 'spGnb')
search_btn.click()

In [4]:
## 최초 검색 시 URL : https://www.jobkorea.co.kr/Search/?stext=%EB%94%A5%EB%9F%AC%EB%8B%9D
## 두번째 페이지 URL : https://www.jobkorea.co.kr/Search/?stext=%EB%94%A5%EB%9F%AC%EB%8B%9D&tabType=recruit&Page_No=2

total_data = []

idx = 1
while True:
    if idx == 6:
        break

    if idx > 1:
        url = f'https://www.jobkorea.co.kr/Search/?stext=%EB%94%A5%EB%9F%AC%EB%8B%9D&tabType=recruit&Page_No={idx}'
        try:
            browser.get(url)
        except:
            print(f'Page Not Found. Current Page No : {idx}')
            break
        
    time.sleep(5)
    container = browser.find_element(By.CLASS_NAME, 'list-default')
    blocks = container.find_elements(By.CLASS_NAME, 'list-post')

    print(f"Page No : {idx}, There are {len(blocks)} job openings.", end=" ")
    for i, block in enumerate(blocks):
        try:
            company_name = block.find_element(By.CLASS_NAME, 'post-list-corp').find_element(By.TAG_NAME, 'a').text
            list_info = block.find_element(By.CLASS_NAME, 'post-list-info').find_element(By.TAG_NAME, 'a')
            title = list_info.get_attribute('title')
            link = list_info.get_attribute('href')
            
            options = block.find_element(By.CLASS_NAME, 'option').find_elements(By.TAG_NAME, 'span')
            options = ','.join([option.text for option in options])

            detail_fields = block.find_element(By.CLASS_NAME, 'etc').text

            data = {'회사명' : company_name, '채용공고 제목' : title, '채용공고 세부 사항' : options, '기술 세부 사항' : detail_fields, '링크' : link}
            total_data.append(data)

        except NoSuchElementException as e:
            print(f'\n\tBlock No : {i}, \n\t{e}')
            

    print(f"--> Current num of data : {len(total_data)}")    

    idx += 1

Page No : 1, There are 20 job openings. --> Current num of data : 20
Page No : 2, There are 20 job openings. --> Current num of data : 40
Page No : 3, There are 20 job openings. --> Current num of data : 60
Page No : 4, There are 20 job openings. --> Current num of data : 80
Page No : 5, There are 20 job openings. --> Current num of data : 100


In [5]:
df = pd.DataFrame(total_data)
df.to_csv("JobKorea_data.csv", encoding='utf-8-sig')
browser.close()